# Retrieval Augmented Generation (RAG) with Langchain
*Using IBM Granite Models*

## In this notebook
This notebook contains instructions for performing Retrieval Augumented Generation (RAG). RAG is an architectural pattern that can be used to augment the performance of language models by recalling factual information from a knowledge base, and adding that information to the model query. The most common approach in RAG is to create dense vector representations of the knowledge base in order to retrieve text chunks that are semantically similar to a given user query.

RAG use cases include:
- Customer service: Answering questions about a product or service using facts from the product documentation.
- Domain knowledge: Exploring a specialized domain (e.g., finance) using facts from papers or articles in the knowledge base.
- News chat: Chatting about current events by calling up relevant recent news articles.

In its simplest form, RAG requires 3 steps:

- Initial setup:
  - Index knowledge-base passages for efficient retrieval. In this recipe, we take embeddings of the passages, and store them in a vector database.
- Upon each user query:
  - Retrieve relevant passages from the database. In this recipe, we use an embedding of the query to retrieve semantically similar passages.
  - Generate a response by feeding retrieved passage into a large language model, along with the user query.

## Setting up the environment

Install dependencies.

In [1]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install git+https://github.com/ibm-granite-community/utils.git \
    transformers \
    langchain_classic \
    langchain_community \
    langchain_text_splitters \
    langchain_huggingface sentence_transformers \
    langchain_milvus 'pymilvus[milvus_lite]' \
    'langchain_replicate @ git+https://github.com/ibm-granite-community/langchain-replicate.git' \
    wget
! echo "::endgroup::"

::group::Install Dependencies
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 92.0 MB/s eta 0:00:00
Using Python 3.12.12 environment at: /usr
Resolved 83 packages in 10.32s
Prepared 17 packages in 2.69s
Uninstalled 2 packages in 13ms
Installed 17 packages in 110ms
 + dataclasses-json==0.6.7
 + ibm-granite-community-utils==0.1.dev123 (from git+https://github.com/ibm-granite-community/utils.git@9058de28a41a3d64ce9bc8c8a6b7a8bee42a30c9)
 + langchain-classic==1.0.0
 + langchain-community==0.4.1
 - langchain-core==1.1.3
 + langchain-core==1.2.0
 + langchain-huggingface==1.2.0
 + langchain-milvus==0.3.1
 + langchain-replicate==0.1.dev26 (from git+https://github.com/ibm-granite-community/langchain-replicate.git@200c6f94a8c3bb59afc5dda0dfd88490cd5ba952)
 + langchain-text-splitters==1.1.0
 + marshmallow==3.26.1
 + milvus-lite==2.5.1
 + mypy-extensions==1.1.0
 + pymilvus==2.6.5
 + replicate==1.0.7
 - requests==2.32.4
 + requests==2.32.5
 + typing-inspect==0.9.0
 + wget==3.2
::endgroup::

## Selecting System Components

### Choose your Embeddings Model

Specify the model to use for generating embedding vectors from text.

To use a model from a provider other than Huggingface, replace this code cell with one from [this Embeddings Model recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Embeddings_Models.ipynb).

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer

embeddings_model_path = "ibm-granite/granite-embedding-30m-english"
embeddings_model = HuggingFaceEmbeddings(
    model_name=embeddings_model_path,
)
embeddings_tokenizer = AutoTokenizer.from_pretrained(embeddings_model_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Choose your Vector Database

Specify the database to use for storing and retrieving embedding vectors.

To connect to a vector database other than Milvus substitute this code cell with one from [this Vector Store recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Vector_Stores.ipynb).

In [3]:
from langchain_milvus import Milvus
import tempfile

db_file = tempfile.NamedTemporaryFile(prefix="milvus_", suffix=".db", delete=False).name
print(f"The vector database will be saved to {db_file}")

vector_db = Milvus(
    embedding_function=embeddings_model,
    connection_args={"uri": db_file},
    auto_id=True,
    index_params={"index_type": "AUTOINDEX"},
)

The vector database will be saved to /tmp/milvus_nxxukhp1.db


### Choose your LLM
The LLM will be used for answering the question, given the retrieved text.

Select a Granite Code model from the [`ibm-granite`](https://replicate.com/ibm-granite) org on Replicate. Here we use the Replicate Langchain client to connect to the model.

To connect to a model on a provider other than Replicate, substitute this code cell with one from the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).

In [5]:
import os
from google.colab import userdata

# --- Set the required API Token Name ---
API_TOKEN_NAME = "REPLICATE_API_TOKEN"

# 1. Retrieve the token value securely from the Colab Secrets panel.
#    You must ensure this key is added to the 'Secrets' tab on the left.
api_key = userdata.get(API_TOKEN_NAME)

# 2. Set the token as a system environment variable.
if api_key:
    os.environ[API_TOKEN_NAME] = api_key
    print(f"✅ SUCCESS: {API_TOKEN_NAME} loaded and set. Models can now authenticate.")
else:
    print(f"❌ ERROR: {API_TOKEN_NAME} not found in Colab Secrets. Please check the Secrets panel.")

# Note: After running this, re-run all subsequent cells that initialize or use the models (like ChatReplicate).

✅ SUCCESS: REPLICATE_API_TOKEN loaded and set. Models can now authenticate.


## Building the Vector Database

In this example, we take the State of the Union speech text, split it into chunks, derive embedding vectors using the embedding model, and load it into the vector database for querying.

### Download the document

Here we use President Biden's State of the Union address from March 1, 2022.

In [6]:
import os
import wget

filename = 'state_of_the_union.txt'
url = 'https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/data/foundation_models/state_of_the_union.txt'

if not os.path.isfile(filename):
  wget.download(url, out=filename)

### Split the document into chunks

Split the document into text segments that can fit into the model's context window.

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=embeddings_tokenizer,
    chunk_size=embeddings_tokenizer.max_len_single_sentence,
    chunk_overlap=0,
)
texts = text_splitter.split_documents(documents)
doc_id = 0
for text in texts:
    text.metadata["doc_id"] = (doc_id:=doc_id+1)
print(f"{len(texts)} text document chunks created")

19 text document chunks created


### Populate the vector database

NOTE: Population of the vector database may take over a minute depending on your embedding model and service.

In [8]:
ids = vector_db.add_documents(texts)
print(f"{len(ids)} documents added to the vector database")

19 documents added to the vector database


## Querying the Vector Database

### Conduct a similarity search

Search the database for similar documents by proximity of the embedded vector in vector space.

In [9]:
query = "What did the president say about Ketanji Brown Jackson?"
docs = vector_db.similarity_search(query)
print(f"{len(docs)} documents returned")
for doc in docs:
    print(doc)
    print("=" * 80)  # Separator for clarity

4 documents returned
page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’

## Answering Questions

### Automate the RAG pipeline

Build a RAG chain with the model and the document retriever.

First we create the prompts for Granite to perform the RAG query. We use the Granite chat template and supply the placeholder values that the LangChain RAG pipeline will replace.

Next, we construct the RAG pipeline by using the Granite prompt templates previously created.

In [11]:
import os
from google.colab import userdata
from langchain_replicate import ChatReplicate
from ibm_granite_community.notebook_utils import get_env_var
from ibm_granite_community.langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

# --- 1. Fix Authentication Error (ReplicateError: Unauthorized) ---
API_TOKEN_NAME = "REPLICATE_API_TOKEN"
api_key = userdata.get(API_TOKEN_NAME)

if api_key:
    # Set the environment variable for both get_env_var and ChatReplicate to use
    os.environ[API_TOKEN_NAME] = api_key
    print(f"✅ SUCCESS: {API_TOKEN_NAME} loaded and set.")
else:
    # This print will alert you if the token is missing in Colab Secrets
    print(f"❌ ERROR: {API_TOKEN_NAME} not found. Please set your token in the Colab Secrets panel.")
    # Stop execution if the token is missing to prevent the Unauthorized error
    # You might need to raise an exception or manually check this step.

# --- 2. Initialize the Language Model (LLM) ---
model_path = "ibm-granite/granite-4.0-h-small"
# The model initialization should now succeed because the token is in os.environ
model = ChatReplicate(
    model=model_path,
    replicate_api_token=get_env_var(API_TOKEN_NAME),
)
print("✅ ChatReplicate model initialized.")


# --- 3. Construct the RAG Chain ---
# NOTE: This step assumes 'vector_db' has been defined in an earlier cell.

# Create a Granite prompt for question-answering with the retrieved context
prompt_template = ChatPromptTemplate.from_template("{input}")

# Assemble the retrieval-augmented generation chain components
combine_docs_chain = create_stuff_documents_chain(
    llm=model,
    prompt=prompt_template,
)

# Final RAG Chain creation
rag_chain = create_retrieval_chain(
    # This requires 'vector_db' to be defined (e.g., as a FAISS index)
    retriever=vector_db.as_retriever(),
    combine_docs_chain=combine_docs_chain,
)
print("✅ RAG Chain created successfully.")

✅ SUCCESS: REPLICATE_API_TOKEN loaded and set.
✅ ChatReplicate model initialized.
✅ RAG Chain created successfully.


### Generate a retrieval-augmented response to a question

Use the RAG chain to process a question. The document chunks relevant to that question are retrieved and used as context.

In [14]:
import os
from google.colab import userdata

# Define the exact environment variable name required by the Replicate/Granite setup
API_TOKEN_NAME = "REPLICATE_API_TOKEN"

# 1. Retrieve the token value securely from the Colab Secrets panel.
#    ***ACTION REQUIRED: You must ensure this key is added to the 'Secrets' tab on the left.***
api_key = userdata.get(API_TOKEN_NAME)

# 2. Set the token as a system environment variable.
if api_key:
    os.environ[API_TOKEN_NAME] = api_key
    print(f"✅ SUCCESS: {API_TOKEN_NAME} loaded and set.")
    print("Please re-run the cells that initialize the 'model' and 'rag_chain'.")
else:
    print(f"❌ ERROR: {API_TOKEN_NAME} not found.")
    print("Please click the 'key' icon (Secrets) on the left panel and add your token.")

✅ SUCCESS: REPLICATE_API_TOKEN loaded and set.
Please re-run the cells that initialize the 'model' and 'rag_chain'.
